In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from pandas.plotting import lag_plot
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa import stattools
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 2000)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/stack-overflow-developer-survey-2020/developer_survey_2020/survey_results_public.csv")


In [ ]:
df.sample(1)

In [ ]:
print(df.shape)
df.info()

In [ ]:
# 
def expand_delimited_col_to_multiple_cols(_df, delimited_col, delimited_sep=';'):
    
    df = _df.copy()
    #
    df[delimited_col] = df[delimited_col].fillna('')
    col = df[delimited_col].str.strip().str.split(delimited_sep)
    
    #
    all_val = []
    for val in col:
        if (len(val) > 0):
            all_val.extend(val )

    all_val = [x for x in all_val if x]
    all_val = np.array(list(set(all_val) )) 
    all_val.sort()
    print('all_val==', all_val)
    
    #
    np_zero_matrix = np.zeros((len(df), len(all_val)))
    np_zero_matrix.shape
    df_dummies = pd.DataFrame(np_zero_matrix, columns=all_val)
    print('df_dummies shape==',df_dummies.shape)

    #
    for i, gen in enumerate( df[delimited_col].fillna('') ):
        indices = df_dummies.columns.get_indexer(df.iloc[i][delimited_col].split(delimited_sep)) 
        if (indices.size > 0):
            df_dummies.iloc[i, indices] = 1

    return df_dummies


In [ ]:
df_DatabaseWorkedWith = expand_delimited_col_to_multiple_cols(df, 'DatabaseWorkedWith')
df_DatabaseWorkedWith.head()
df_sum_DatabaseWorkedWith = df_DatabaseWorkedWith.sum(axis=0).sort_values(ascending=False).to_frame( name='DatabaseWorkedWith')
df_sum_DatabaseWorkedWith


In [ ]:
df_DatabaseDesireNextYear = expand_delimited_col_to_multiple_cols(df, 'DatabaseDesireNextYear')
df_DatabaseDesireNextYear.head()
df_sum_DatabaseDesireNextYear = df_DatabaseDesireNextYear.sum(axis=0).sort_values(ascending=False).to_frame(name='DatabaseDesireNextYear') 
df_sum_DatabaseDesireNextYear


In [ ]:
df_compare_Database = df_sum_DatabaseWorkedWith.merge(right=df_sum_DatabaseDesireNextYear, how='outer', left_index=True, right_index=True)
df_compare_Database = df_compare_Database.sort_values(by=['DatabaseWorkedWith','DatabaseDesireNextYear'], ascending=[False,False])
print(df_compare_Database)
df_compare_Database.plot.barh().invert_yaxis()
plt.tight_layout()


In [ ]:
df_LanguageWorkedWith = expand_delimited_col_to_multiple_cols(df, 'LanguageWorkedWith')
df_LanguageWorkedWith.head()   
df_sum_LanguageWorkedWith = df_LanguageWorkedWith.sum(axis=0).sort_values(ascending=False).to_frame(name='LanguageWorkedWith') 
df_sum_LanguageWorkedWith


In [ ]:
df_LanguageDesireNextYear = expand_delimited_col_to_multiple_cols(df, 'LanguageDesireNextYear')
df_LanguageDesireNextYear.head()
df_sum_LanguageDesireNextYear = df_LanguageDesireNextYear.sum(axis=0).sort_values(ascending=False).to_frame(name='LanguageDesireNextYear') 
df_sum_LanguageDesireNextYear


In [ ]:
df_compare_Language = df_sum_LanguageWorkedWith.merge(right=df_sum_LanguageDesireNextYear, how='outer', left_index=True, right_index=True)
df_compare_Language = df_compare_Language.sort_values(by=['LanguageWorkedWith','LanguageDesireNextYear'], ascending=[False,False])
print(df_compare_Language)
df_compare_Language.plot.barh().invert_yaxis()
plt.tight_layout()

In [ ]:
df_WebframeWorkedWith = expand_delimited_col_to_multiple_cols(df, 'WebframeWorkedWith')
df_WebframeWorkedWith.head() 
df_sum_WebframeWorkedWith = df_WebframeWorkedWith.sum(axis=0).sort_values(ascending=False).to_frame(name='WebframeWorkedWith') 
df_sum_WebframeWorkedWith


In [ ]:
df_WebframeDesireNextYear = expand_delimited_col_to_multiple_cols(df, 'WebframeDesireNextYear')
df_WebframeDesireNextYear.head() 
df_sum_WebframeDesireNextYear = df_WebframeDesireNextYear.sum(axis=0).sort_values(ascending=False).to_frame(name='WebframeDesireNextYear') 
df_sum_WebframeDesireNextYear 


In [ ]:
df_compare_Webframe = df_sum_WebframeWorkedWith.merge(right=df_sum_WebframeDesireNextYear, how='outer', left_index=True, right_index=True)
df_compare_Webframe = df_compare_Webframe.sort_values(by=['WebframeWorkedWith','WebframeDesireNextYear'], ascending=[False,False])
print(df_compare_Webframe)
df_compare_Webframe.plot.barh().invert_yaxis()
plt.tight_layout()

In [ ]:
df_PlatformWorkedWith = expand_delimited_col_to_multiple_cols(df, 'PlatformWorkedWith')
df_PlatformWorkedWith.head() 
df_sum_PlatformWorkedWith = df_PlatformWorkedWith.sum(axis=0).sort_values(ascending=False).to_frame(name='PlatformWorkedWith') 
df_sum_PlatformWorkedWith


In [ ]:
df_PlatformDesireNextYear = expand_delimited_col_to_multiple_cols(df, 'PlatformDesireNextYear')
df_PlatformDesireNextYear.head() 
df_sum_PlatformDesireNextYear = df_PlatformDesireNextYear.sum(axis=0).sort_values(ascending=False).to_frame(name='PlatformDesireNextYear') 
df_sum_PlatformDesireNextYear




In [ ]:
df_compare_Platform = df_sum_PlatformWorkedWith.merge(right=df_sum_PlatformDesireNextYear, how='outer', left_index=True, right_index=True)
df_compare_Platform.sort_values(by=['PlatformWorkedWith','PlatformDesireNextYear'], ascending=[False,False])
 

In [ ]:
df_MiscTechWorkedWith = expand_delimited_col_to_multiple_cols(df, 'MiscTechWorkedWith')
df_MiscTechWorkedWith.head() 
df_sum_MiscTechWorkedWith = df_MiscTechWorkedWith.sum(axis=0).sort_values(ascending=False).to_frame(name='MiscTechWorkedWith') 
df_sum_MiscTechWorkedWith


In [ ]:
df_MiscTechDesireNextYear = expand_delimited_col_to_multiple_cols(df, 'MiscTechDesireNextYear')
df_MiscTechDesireNextYear.head()
df_sum_MiscTechDesireNextYear = df_MiscTechDesireNextYear.sum(axis=0).sort_values(ascending=False).to_frame(name='MiscTechDesireNextYear') 
df_sum_MiscTechDesireNextYear


In [ ]:
df_compare_MiscTech = df_sum_MiscTechWorkedWith.merge(right=df_sum_MiscTechDesireNextYear, how='outer', left_index=True, right_index=True)
df_compare_MiscTech = df_compare_MiscTech.sort_values(by=['MiscTechWorkedWith','MiscTechDesireNextYear'], ascending=[False,False])
print(df_compare_MiscTech)
df_compare_MiscTech.plot.barh().invert_yaxis()
plt.tight_layout()


In [ ]:
sns.jointplot(x='Age', y='WorkWeekHrs', data=df.loc[df['WorkWeekHrs'] < 100 , :],  xlim=(0, 100))


In [ ]:
df.sample(1)

In [ ]:
df['Employment'].value_counts().sort_values(ascending=False)
 

In [ ]:
df_DevType = expand_delimited_col_to_multiple_cols(df, 'DevType')
df_DevType.head()
df_sum_DevType = df_DevType.sum(axis=0).sort_values(ascending=False).to_frame(name='DevType') 
df_sum_DevType


In [ ]:
df['MainBranch'].value_counts().sort_values(ascending=False)


In [ ]:
df['JobSat'].value_counts().sort_values(ascending=False)


In [ ]:
df['EdLevel'].value_counts().sort_values(ascending=False)


In [ ]:
df['UndergradMajor'].value_counts().sort_values(ascending=False)


In [ ]:
df['JobSeek'].value_counts().sort_values(ascending=False) / df['JobSeek'].count()


In [ ]:
df_JobFactors = expand_delimited_col_to_multiple_cols(df, 'JobFactors')
df_JobFactors.head() 
df_sum_JobFactors = df_JobFactors.sum(axis=0).sort_values(ascending=False).to_frame(name='JobFactors') 
df_sum_JobFactors


In [ ]:
df_NEWJobHuntResearch = expand_delimited_col_to_multiple_cols(df, 'NEWJobHuntResearch')
df_NEWJobHuntResearch.head() 
df_sum_NEWJobHuntResearch = df_NEWJobHuntResearch.sum(axis=0).sort_values(ascending=False).to_frame(name='NEWJobHuntResearch') 
df_sum_NEWJobHuntResearch


In [ ]:
df_NEWJobHunt = expand_delimited_col_to_multiple_cols(df, 'NEWJobHunt')
df_NEWJobHunt.head() 
df_sum_NEWJobHunt = df_NEWJobHunt.sum(axis=0).sort_values(ascending=False).to_frame(name='NEWJobHunt') 
df_sum_NEWJobHunt


In [ ]:
df_NEWOvertime = expand_delimited_col_to_multiple_cols(df, 'NEWOvertime')
df_NEWOvertime.head() 
df_sum_NEWOvertime = df_NEWOvertime.sum(axis=0).sort_values(ascending=False).to_frame(name='NEWOvertime') 
df_sum_NEWOvertime


In [ ]:
df_NEWStuck = expand_delimited_col_to_multiple_cols(df, 'NEWStuck')
df_NEWStuck.head() 
df_sum_NEWStuck = df_NEWStuck.sum(axis=0).sort_values(ascending=False).to_frame(name='NEWStuck') 
df_sum_NEWStuck



In [ ]:
df_NEWPurchaseResearch = expand_delimited_col_to_multiple_cols(df, 'NEWPurchaseResearch')
df_NEWPurchaseResearch.head() 
df_sum_NEWPurchaseResearch = df_NEWPurchaseResearch.sum(axis=0).sort_values(ascending=False).to_frame(name='NEWPurchaseResearch') 
df_sum_NEWPurchaseResearch
 

In [ ]:
df_NEWSOSites = expand_delimited_col_to_multiple_cols(df, 'NEWSOSites')
df_NEWSOSites.head() 
df_sum_NEWSOSites = df_NEWSOSites.sum(axis=0).sort_values(ascending=False).to_frame(name='NEWSOSites') 
df_sum_NEWSOSites

In [ ]:
df.sample(1)



In [ ]:
fig,ax=plt.subplots(figsize=(10,5))
df.Age.dropna().plot(kind="hist", ax=ax, alpha=.5, bins=100, density=True )
ax.set_xlim(0, 100)


In [ ]:
df.YearsCode.sort_values().unique()
    
YearsCode_categories = np.array(['Less than 1 year'] + [str(x) for x in np.arange(1,51) ] + ['More than 50 years', 'MISSING'])
df['YearsCode_categories'] = pd.Categorical(df["YearsCode"].fillna("MISSING"), categories=YearsCode_categories )
 
    
plt.figure(figsize=(6,10))  
sns.countplot(data=df , y="YearsCode_categories")
plt.tight_layout()



In [ ]:
 
sns.stripplot(y='JobSat',x='WorkWeekHrs', data=df.sample(10000)  ).set_xlim(0,100)
 


In [ ]:
#df['OrgSize'].value_counts().sort_values(ascending=False).plot()

df['WorkWeekHrs'].value_counts().sort_values(ascending=False).plot()
pd.cut(df['WorkWeekHrs'], np.arange(0,101,5) ).value_counts().sort_values(ascending=False)

df['DatabaseDesireNextYear'], df['DatabaseWorkedWith']

#pd.crosstab(tips['day'], tips['size'])

